In [9]:
# entire code is being run on an Ubuntu kernel. 
# not many QoL libraries are needed.
# your mileage may vary.
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# training specific imports
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold

# dataset imports
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torchvision.transforms import ToTensor, Resize, ToPILImage, Lambda
import torchvision.transforms as transforms

# AI imports
import torch
import torch.nn as nn

# statistics
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import precision_recall_fscore_support, roc_curve

# test if GPU acceleration is working
torch.cuda.is_available()

True

## This notebook will be used for the AI part of the project, since the preprocessing step took a bit too long

In [10]:
# load clean csv files in
train = pd.read_csv('./disaster-dataset/train.csv')
test = pd.read_csv('./disaster-dataset/test.csv')
train.shape, test.shape

((14920, 27), (3365, 27))

In [11]:
train.head()

,Filename,Sentiment polarity,Stimulation and excitement,Joy,Sadness,Fear,Disgust,Anger,Surprise,Neutral,...,Relief,Sadness.1,Surprise.1,Human facial expression gesture,Inherent image property,Image background,Objects in image,Text in image,Emoji sticker,Halo effect
0,052464df-5fdb-428a-9a4a-6036ea023b42.jpg,0.375,0.75,0,0,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0
1,2ce9b1bc-ee70-4128-85d1-0fd3ac272619.jpg,0.625,0.50,0,0,0,0,0,0,1,...,1,0,0,1,0,0,0,0,0,0
2,2280b9f3-6989-4222-a0d9-ec4bd35d731b.jpg,0.750,1.00,1,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,257a8e84-7738-49e2-9945-7f082bf7ad34.jpg,0.000,0.50,0,1,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
4,3f6b222a-049c-4803-9cbb-5ea734110ed0.jpg,0.000,1.00,0,1,1,0,0,0,0,...,0,1,0,1,0,1,1,0,0,0


In [12]:
test.head()

,Filename,Sentiment polarity,Stimulation and excitement,Joy,Sadness,Fear,Disgust,Anger,Surprise,Neutral,...,Relief,Sadness.1,Surprise.1,Human facial expression gesture,Inherent image property,Image background,Objects in image,Text in image,Emoji sticker,Halo effect
0,e991e17b-1e4b-43e6-a05d-a025d7e1c20a.jpg,0.375,0.875,0,0,1,0,0,1,0,...,0,0,1,0,1,1,0,0,0,1
1,4f89d359-6788-4b71-ad1a-1f3cd95a16ef.jpg,0.000,1.000,0,0,1,0,0,0,0,...,0,1,0,0,1,1,0,0,0,0
2,f2f954f0-287a-423b-8c3f-21e7c4ed6f62.jpg,0.000,0.500,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,954a5e15-f99b-4e2f-8d71-90cc9911f468.jpg,0.125,0.750,0,1,0,1,1,0,0,...,1,1,0,1,0,0,0,0,0,1
4,e1a71c6b-a899-4e11-8cfc-cbcd23865ee1.jpg,0.250,1.000,0,1,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0


In [13]:
# rewrite custom config for dataset class and loaders
# Recreate a custom dataset class.
class SentimentDataset(Dataset):

    """A custom dataset class created for the sentiment analysis portion of project Ayre.
    NOTE: Make sure the annotations entry is the loaded dataframe itself and NOT the filepath.
    This was done to smoothen our own training process."""

    def __init__(self, annotations: pd.DataFrame, img_dir: str, transform=None, target_transform=None):
        self.img_labels = annotations
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.target_names = self.img_labels.columns[1:]

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1:]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, torch.tensor(label)

In [14]:
# initialize all dataset params
root = './disaster-dataset/images/'
img_w = 512
img_h = 512
train_batch = 96
test_batch = 64

# creating a custom transform
preproc = transforms.Compose([
    ToPILImage(),
    Resize((img_h, img_w), antialias=True),
    Lambda(lambda img: img.convert('RGB')),
    ToTensor(),
])

In [ ]:
# create dataset objects
train_dataset = SentimentDataset(train, root, transform=preproc)
test_dataset = SentimentDataset(test, root, transform=preproc)

In [ ]:
# create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=train_batch, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=test_batch, shuffle=True)

### AI work begins here.